In [ ]:
import numpy as np
import pandas as pd
import os
import random
import copy
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from geopy.distance import geodesic

import matplotlib.cbook as cbook
from google.colab import drive

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import r2_score
import numpy as np

import tensorflow as tf
from keras.metrics import Accuracy, Precision, Recall, AUC, BinaryAccuracy

drive.mount('/content/drive')

print(pd. __version__)

Mounted at /content/drive
2.1.4


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Hoora/Cache Money - Hoora x Jagdesh -20240713T075605Z-001/Cache Money - Hoora x Jagdesh/all_waybill_info_meituan.csv')
df.head()

,Unnamed: 0,dt,order_id,waybill_id,courier_id,da_id,is_courier_grabbed,is_weekend,estimate_arrived_time,is_prebook,poi_id,sender_lng,sender_lat,recipient_lng,recipient_lat,grab_lng,grab_lat,dispatch_time,grab_time,fetch_time,estimate_meal_prepare_time,arrive_time,order_push_time,platform_order_time
0,0,20221017,0,0,0,0,0,1,1665937978,0,0,174579111,45862631,174600135,45852786,0,0,1665936258,0,0,1665935996,0,1665936000,1665935996
1,1,20221017,1,1,1,1,1,1,1665937864,0,1,174529930,45905850,174522983,45898250,174530062,45906005,1665936734,1665936737,1665937344,1665935995,1665937798,1665936006,1665935995
2,2,20221017,2,2,2,2,1,0,1665939504,0,2,174532407,45861460,174547033,45891243,174548244,45870923,1665937484,1665937501,1665938346,1665937097,1665939384,1665937107,1665937097
3,3,20221017,3,3,3,0,1,0,1665939432,0,3,174559718,45868015,174595327,45886787,174560199,45867948,1665937394,1665937397,1665937830,1665937028,1665939822,1665937369,1665937028
4,4,20221017,4,4,4,0,1,0,1665938894,0,3,174559718,45868015,174534558,45867411,174554896,45865167,1665937394,1665937413,1665937710,1665937092,1665938262,1665937373,1665937092


In [ ]:
def convert_unix_time(_df, col):
  _df[col] = pd.to_datetime(_df[col], unit='s')
  _df[col] = _df[col].dt.strftime('%Y-%m-%d %H:%M:%S')
  _df[col] = pd.to_datetime(_df[col], format='%Y-%m-%d %H:%M:%S')
  _df[col + '_minute'] = _df[col].dt.minute

In [ ]:
_df = df.copy()
_df['dt'] = pd.to_datetime(_df['dt'], format='%Y%m%d')
#
convert_unix_time(_df, 'estimate_arrived_time')
convert_unix_time(_df, 'platform_order_time')
convert_unix_time(_df, 'order_push_time')
convert_unix_time(_df, 'arrive_time')
convert_unix_time(_df, 'estimate_meal_prepare_time')
convert_unix_time(_df, 'fetch_time')
convert_unix_time(_df, 'grab_time')
convert_unix_time(_df, 'dispatch_time')

_df = _df[(_df['arrive_time'].dt.year != 1970)]

In [ ]:
_df['PAV_Diff'] = _df['estimate_arrived_time'] - _df['arrive_time']
# Get the difference in minutes
_df['PAV_CONSTRAINT_MINS'] = np.round(_df['PAV_Diff'].dt.total_seconds() / 60,  2)
_df['PAV_CONSTRAINT'] = np.where(abs(_df['PAV_CONSTRAINT_MINS']) > 5, 1, 0) ## Minute_Constraint

<ipython-input-5-e46b9524e6db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['PAV_Diff'] = _df['estimate_arrived_time'] - _df['arrive_time']
<ipython-input-5-e46b9524e6db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['PAV_CONSTRAINT_MINS'] = np.round(_df['PAV_Diff'].dt.total_seconds() / 60,  2)


In [ ]:
_df.head()

,Unnamed: 0,dt,order_id,waybill_id,courier_id,da_id,is_courier_grabbed,is_weekend,estimate_arrived_time,is_prebook,poi_id,sender_lng,sender_lat,recipient_lng,recipient_lat,grab_lng,grab_lat,dispatch_time,grab_time,fetch_time,estimate_meal_prepare_time,arrive_time,order_push_time,platform_order_time,estimate_arrived_time_minute,platform_order_time_minute,order_push_time_minute,arrive_time_minute,estimate_meal_prepare_time_minute,fetch_time_minute,grab_time_minute,dispatch_time_minute,PAV_Diff,PAV_CONSTRAINT_MINS,PAV_CONSTRAINT
1,1,2022-10-17,1,1,1,1,1,1,2022-10-16 16:31:04,0,1,174529930,45905850,174522983,45898250,174530062,45906005,2022-10-16 16:12:14,2022-10-16 16:12:17,2022-10-16 16:22:24,2022-10-16 15:59:55,2022-10-16 16:29:58,2022-10-16 16:00:06,2022-10-16 15:59:55,31,59,0,29,59,22,12,12,0 days 00:01:06,1.10,0
2,2,2022-10-17,2,2,2,2,1,0,2022-10-16 16:58:24,0,2,174532407,45861460,174547033,45891243,174548244,45870923,2022-10-16 16:24:44,2022-10-16 16:25:01,2022-10-16 16:39:06,2022-10-16 16:18:17,2022-10-16 16:56:24,2022-10-16 16:18:27,2022-10-16 16:18:17,58,18,18,56,18,39,25,24,0 days 00:02:00,2.00,0
3,3,2022-10-17,3,3,3,0,1,0,2022-10-16 16:57:12,0,3,174559718,45868015,174595327,45886787,174560199,45867948,2022-10-16 16:23:14,2022-10-16 16:23:17,2022-10-16 16:30:30,2022-10-16 16:17:08,2022-10-16 17:03:42,2022-10-16 16:22:49,2022-10-16 16:17:08,57,17,22,3,17,30,23,23,-1 days +23:53:30,-6.50,1
4,4,2022-10-17,4,4,4,0,1,0,2022-10-16 16:48:14,0,3,174559718,45868015,174534558,45867411,174554896,45865167,2022-10-16 16:23:14,2022-10-16 16:23:33,2022-10-16 16:28:30,2022-10-16 16:18:12,2022-10-16 16:37:42,2022-10-16 16:22:53,2022-10-16 16:18:12,48,18,22,37,18,28,23,23,0 days 00:10:32,10.53,1
5,5,2022-10-17,5,5,5,3,1,0,2022-10-16 17:15:00,0,4,174542688,45920329,174530185,45921310,174526338,45912309,2022-10-16 16:45:27,2022-10-16 16:45:41,2022-10-16 17:07:41,2022-10-16 16:39:59,2022-10-16 17:14:56,2022-10-16 16:40:03,2022-10-16 16:39:59,15,39,40,14,39,7,45,45,0 days 00:00:04,0.07,0


In [ ]:
# _df['PAV_Diff'] = _df['estimate_arrived_time_minute'] - _df['arrive_time_minute']
# _df['PAV_CONSTRAINT'] = np.where(abs(_df['PAV_Diff']) > 20, 1, 0) ## Minute_Constraint

In [ ]:
# Convert long format to proper lat/lng by dividing by 1e6
_df['sender_lng'] = _df['sender_lng'] / 1e6
_df['sender_lat'] = _df['sender_lat'] / 1e6
_df['recipient_lng'] = _df['recipient_lng'] / 1e6
_df['recipient_lat'] = _df['recipient_lat'] / 1e6

# Function to calculate the distance
def calculate_distance(row):
    sender_coords = (row['sender_lat'], row['sender_lng'])
    recipient_coords = (row['recipient_lat'], row['recipient_lng'])
    return geodesic(sender_coords, recipient_coords).miles

# Apply the function to calculate the distance
_df['distance_miles'] = _df.apply(calculate_distance, axis=1)


In [ ]:
_df['distance_miles'].min()

0.0

In [ ]:
_df['distance_miles'].max()

11.672473342396607

In [ ]:
# Apply the distance constraint (30 miles)
_df['DISTANCE_CONSTRAINT'] = np.where(_df['distance_miles'] > 2, 1, 0) ## Distance_Constraint

In [ ]:
# _df['GP/UNGP'] = np.where(((_df['PAV_CONSTRAINT'] == 0) & (_df['DISTANCE_CONSTRAINT'] == 0)),0, 1) ## Grouping

_df['GP/UNGP'] = np.where(((abs(_df['PAV_CONSTRAINT_MINS'] > 5)) & (_df['distance_miles'] > 2)), 1, 0) ## Grouping

In [ ]:
_df.head()

,Unnamed: 0,dt,order_id,waybill_id,courier_id,da_id,is_courier_grabbed,is_weekend,estimate_arrived_time,is_prebook,poi_id,sender_lng,sender_lat,recipient_lng,recipient_lat,grab_lng,grab_lat,dispatch_time,grab_time,fetch_time,estimate_meal_prepare_time,arrive_time,order_push_time,platform_order_time,estimate_arrived_time_minute,platform_order_time_minute,order_push_time_minute,arrive_time_minute,estimate_meal_prepare_time_minute,fetch_time_minute,grab_time_minute,dispatch_time_minute,PAV_Diff,PAV_CONSTRAINT_MINS,PAV_CONSTRAINT,distance_miles,DISTANCE_CONSTRAINT,GP/UNGP
1,1,2022-10-17,1,1,1,1,1,1,2022-10-16 16:31:04,0,1,174.529930,45.905850,174.522983,45.898250,174530062,45906005,2022-10-16 16:12:14,2022-10-16 16:12:17,2022-10-16 16:22:24,2022-10-16 15:59:55,2022-10-16 16:29:58,2022-10-16 16:00:06,2022-10-16 15:59:55,31,59,0,29,59,22,12,12,0 days 00:01:06,1.10,0,0.622672,0,0
2,2,2022-10-17,2,2,2,2,1,0,2022-10-16 16:58:24,0,2,174.532407,45.861460,174.547033,45.891243,174548244,45870923,2022-10-16 16:24:44,2022-10-16 16:25:01,2022-10-16 16:39:06,2022-10-16 16:18:17,2022-10-16 16:56:24,2022-10-16 16:18:27,2022-10-16 16:18:17,58,18,18,56,18,39,25,24,0 days 00:02:00,2.00,0,2.174600,1,0
3,3,2022-10-17,3,3,3,0,1,0,2022-10-16 16:57:12,0,3,174.559718,45.868015,174.595327,45.886787,174560199,45867948,2022-10-16 16:23:14,2022-10-16 16:23:17,2022-10-16 16:30:30,2022-10-16 16:17:08,2022-10-16 17:03:42,2022-10-16 16:22:49,2022-10-16 16:17:08,57,17,22,3,17,30,23,23,-1 days +23:53:30,-6.50,1,2.152113,1,0
4,4,2022-10-17,4,4,4,0,1,0,2022-10-16 16:48:14,0,3,174.559718,45.868015,174.534558,45.867411,174554896,45865167,2022-10-16 16:23:14,2022-10-16 16:23:33,2022-10-16 16:28:30,2022-10-16 16:18:12,2022-10-16 16:37:42,2022-10-16 16:22:53,2022-10-16 16:18:12,48,18,22,37,18,28,23,23,0 days 00:10:32,10.53,1,1.214637,0,0
5,5,2022-10-17,5,5,5,3,1,0,2022-10-16 17:15:00,0,4,174.542688,45.920329,174.530185,45.921310,174526338,45912309,2022-10-16 16:45:27,2022-10-16 16:45:41,2022-10-16 17:07:41,2022-10-16 16:39:59,2022-10-16 17:14:56,2022-10-16 16:40:03,2022-10-16 16:39:59,15,39,40,14,39,7,45,45,0 days 00:00:04,0.07,0,0.606467,0,0


In [ ]:
_df['GP/UNGP'].value_counts()

GP/UNGP
0    549163
1     19382
Name: count, dtype: int64

In [ ]:
# _df['dt'].unique()

In [ ]:
# estimate_meal_prepare_time,'is_weekend','is_courier_grabbed' estimate_arrived_time, arrive_time, sender_lng	sender_lat	recipient_lng	recipient_lat

In [ ]:
# df_model = _df[['estimate_meal_prepare_time_minute','is_weekend','is_courier_grabbed', 'PAV_CONSTRAINT_MINS', 'PAV_CONSTRAINT', 'distance_miles', 	'DISTANCE_CONSTRAINT', 'GP/UNGP']]
# df_model = _df[['estimate_meal_prepare_time_minute','PAV_CONSTRAINT_MINS', 'distance_miles', 'GP/UNGP']]
df_model = _df[['estimate_meal_prepare_time_minute','is_weekend','is_courier_grabbed','PAV_CONSTRAINT_MINS', 'distance_miles', 'GP/UNGP']]
df_model.drop_duplicates(inplace=True)
df_model

<ipython-input-17-678c21db0051>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.drop_duplicates(inplace=True)


,estimate_meal_prepare_time_minute,is_weekend,is_courier_grabbed,PAV_CONSTRAINT_MINS,distance_miles,GP/UNGP
1,59,1,1,1.10,0.622672,0
2,18,0,1,2.00,2.174600,0
3,17,0,1,-6.50,2.152113,0
4,18,0,1,10.53,1.214637,0
5,39,0,1,0.07,0.606467,0
...,...,...,...,...,...,...
654337,39,0,1,8.28,0.217602,0
654339,55,0,1,9.17,0.421045,0
654340,22,0,1,3.67,1.309213,0
654341,49,0,1,3.58,0.706543,0


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from imblearn.over_sampling import SMOTE



X = df_model.drop('GP/UNGP', axis=1)
y = df_model['GP/UNGP']


# Initialize SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X, y)


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_res, y_train_res, test_size=0.3, random_state=54)



# # Standardize the features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


In [ ]:
# X_train

In [ ]:
len(X_train), len(X_test)

(768819, 329495)

In [ ]:
len(X)

568539

In [ ]:
len(X_train) > len(X_test)

True

In [ ]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Use sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
24026/24026 [==============================] - 69s 3ms/step - loss: 0.0384 - accuracy: 0.9857 - val_loss: 0.0263 - val_accuracy: 0.9923
Epoch 2/10
24026/24026 [==============================] - 71s 3ms/step - loss: 0.0186 - accuracy: 0.9941 - val_loss: 0.0118 - val_accuracy: 0.9951
Epoch 3/10
24026/24026 [==============================] - 65s 3ms/step - loss: 0.0160 - accuracy: 0.9948 - val_loss: 0.0180 - val_accuracy: 0.9936
Epoch 4/10
24026/24026 [==============================] - 63s 3ms/step - loss: 0.0148 - accuracy: 0.9953 - val_loss: 0.0048 - val_accuracy: 0.9982
Epoch 5/10
24026/24026 [==============================] - 64s 3ms/step - loss: 0.0141 - accuracy: 0.9955 - val_loss: 0.0041 - val_accuracy: 0.9986
Epoch 6/10
24026/24026 [==============================] - 71s 3ms/step - loss: 0.0130 - accuracy: 0.9958 - val_loss: 0.0058 - val_accuracy: 0.9974
Epoch 7/10
24026/24026 [==============================] - 62s 3ms/step - loss: 0.0123 - accuracy: 0.9960 - val_loss: 0

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Make classify
classification = (model.predict(X_test) > 0.5).astype("int32")
print(classification)

10297/10297 [==============================] - 16s 2ms/step - loss: 0.0106 - accuracy: 0.9963
Test Accuracy: 0.9963
10297/10297 [==============================] - 15s 1ms/step
[[0]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict on the validation set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # For multi-class classification

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

print("Classification Report:")
print(classification_report(y_test, y_pred_classes))


10297/10297 [==============================] - 15s 1ms/step
Confusion Matrix:
[[164844      0]
 [164651      0]]
Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.50      1.00      0.67    164844
           1       0.00      0.00      0.00    164651

    accuracy                           0.50    329495
   macro avg       0.25      0.50      0.33    329495
weighted avg       0.25      0.50      0.33    329495



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Predict and evaluate
y_pred_rf = rf.predict(X_test)
print("Random Forest:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest:
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     48304
           1       1.00      1.00      1.00    122260

    accuracy                           1.00    170564
   macro avg       1.00      1.00      1.00    170564
weighted avg       1.00      1.00      1.00    170564



In [ ]:
from sklearn.model_selection import cross_val_score

# Initialize the Random Forest model
rf = RandomForestClassifier()

# Perform cross-validation
scores = cross_val_score(rf, X, y, cv=5)  # 5-fold cross-validation

print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", np.mean(scores))


Cross-Validation Scores: [1. 1. 1. 1. 1.]
Mean Accuracy: 1.0
